##**4. LSTM, GRU**
1. 기존 RNN과 다른 부분에 대해서 배웁니다.
2. 이전 실습에 이어 다양한 적용법을 배웁니다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **필요 패키지 import**

In [2]:
from tqdm import tqdm
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import torch

### **데이터 전처리**

아래의 sample data를 확인해봅시다.  
이전 실습과 동일합니다.

In [3]:
vocab_size = 100
pad_id = 0

data = [
  [85,14,80,34,99,20,31,65,53,86,3,58,30,4,11,6,50,71,74,13],
  [62,76,79,66,32],
  [93,77,16,67,46,74,24,70],
  [19,83,88,22,57,40,75,82,4,46],
  [70,28,30,24,76,84,92,76,77,51,7,20,82,94,57],
  [58,13,40,61,88,18,92,89,8,14,61,67,49,59,45,12,47,5],
  [22,5,21,84,39,6,9,84,36,59,32,30,69,70,82,56,1],
  [94,21,79,24,3,86],
  [80,80,33,63,34,63],
  [87,32,79,65,2,96,43,80,85,20,41,52,95,50,35,96,24,80]
]

In [4]:
max_len = len(max(data, key=len))
print(f"Maximum sequence length: {max_len}")

valid_lens = []
for i, seq in enumerate(tqdm(data)):
  valid_lens.append(len(seq))
  if len(seq) < max_len:
    data[i] = seq + [pad_id] * (max_len - len(seq))

100%|██████████| 10/10 [00:00<00:00, 8333.61it/s]

Maximum sequence length: 20


In [5]:
# B: batch size, L: maximum sequence length
batch = torch.LongTensor(data)  # (B, L)
batch_lens = torch.LongTensor(valid_lens)  # (B)

batch_lens, sorted_idx = batch_lens.sort(descending=True)
batch = batch[sorted_idx]

print(batch)
print(batch_lens)

tensor([[85, 14, 80, 34, 99, 20, 31, 65, 53, 86,  3, 58, 30,  4, 11,  6, 50, 71,
         74, 13],
        [58, 13, 40, 61, 88, 18, 92, 89,  8, 14, 61, 67, 49, 59, 45, 12, 47,  5,
          0,  0],
        [87, 32, 79, 65,  2, 96, 43, 80, 85, 20, 41, 52, 95, 50, 35, 96, 24, 80,
          0,  0],
        [22,  5, 21, 84, 39,  6,  9, 84, 36, 59, 32, 30, 69, 70, 82, 56,  1,  0,
          0,  0],
        [70, 28, 30, 24, 76, 84, 92, 76, 77, 51,  7, 20, 82, 94, 57,  0,  0,  0,
          0,  0],
        [19, 83, 88, 22, 57, 40, 75, 82,  4, 46,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [93, 77, 16, 67, 46, 74, 24, 70,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [94, 21, 79, 24,  3, 86,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [80, 80, 33, 63, 34, 63,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [62, 76, 79, 66, 32,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0]])
tensor([2

### **LSTM 사용**

LSTM에선 cell state가 추가됩니다.  
Cell state의 shape는 hidden state의 그것과 동일합니다.

In [6]:
embedding_size = 256
hidden_size = 512
num_layers = 1
num_dirs = 1

embedding = nn.Embedding(vocab_size, embedding_size)
lstm = nn.LSTM(
    input_size=embedding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    bidirectional=True if num_dirs > 1 else False
)

h_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (num_layers * num_dirs, B, d_h)
c_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (num_layers * num_dirs, B, d_h)

In [7]:
# d_w: word embedding size
batch_emb = embedding(batch)  # (B, L, d_w)

packed_batch = pack_padded_sequence(batch_emb.transpose(0, 1), batch_lens)

packed_outputs, (h_n, c_n) = lstm(packed_batch, (h_0, c_0))
print(packed_outputs)
print(packed_outputs[0].shape)
print(h_n.shape)
print(c_n.shape)

PackedSequence(data=tensor([[ 0.0297, -0.0759,  0.1151,  ..., -0.0262, -0.1328, -0.1900],
        [-0.0824, -0.0088, -0.1594,  ...,  0.0691,  0.0334, -0.1163],
        [-0.0782,  0.1323,  0.0554,  ...,  0.0391, -0.0544, -0.1738],
        ...,
        [ 0.0439,  0.1663, -0.0978,  ..., -0.0538,  0.0549, -0.0498],
        [ 0.0488,  0.0062, -0.0236,  ..., -0.0676,  0.1770, -0.0364],
        [-0.0587, -0.0118,  0.0083,  ..., -0.0900,  0.0264,  0.0454]],
       grad_fn=<CatBackward>), batch_sizes=tensor([10, 10, 10, 10, 10,  9,  7,  7,  6,  6,  5,  5,  5,  5,  5,  4,  4,  3,
         1,  1]), sorted_indices=None, unsorted_indices=None)
torch.Size([123, 512])
torch.Size([1, 10, 512])
torch.Size([1, 10, 512])


In [8]:
# packed output을 원래 모양으로

outputs, output_lens = pad_packed_sequence(packed_outputs)
print(outputs.shape)
print(output_lens)

torch.Size([20, 10, 512])
tensor([20, 18, 18, 17, 15, 10,  8,  6,  6,  5])


### **GRU 사용**

GRU는 cell state가 없어 RNN과 동일하게 사용 가능합니다.   
GRU를 이용하여 LM task를 수행해봅시다.
- LM : Language Modeling
- 전체 sequence를 통으로 RNN에 넣지 말고 순차적으로 한 time step씩 생성해 다음 time step에 넣어줘야 한다
- for loop을 돌아야 한다

In [9]:
gru = nn.GRU(
    input_size=embedding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    bidirectional=True if num_dirs > 1 else False
)

In [10]:
output_layer = nn.Linear(hidden_size, vocab_size)

# 가장 큰 값 가지는 index 추출
# vocab 내에서 정답으로 선택될 확률 높은 단어

In [11]:
# input_id : 처음 단어
# 처음에는 단어 하나로 시작
input_id = batch.transpose(0, 1)[0, :]  # (B)
hidden = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (1, B, d_h)

Teacher forcing 없이 이전에 얻은 결과를 다음 input으로 이용합니다.
- Teacher forcing : 처음에는 이전에 생성된 단어들이 아닌 GT 단어들을 넣어준다

- 모델이 이 단어들을 생성했다고 가정하고 cheating을 해준다

In [12]:
for t in range(max_len):
  # unsqueeze -> 모양을 (1, )으로 맞춰주기 위해
  input_emb = embedding(input_id).unsqueeze(0)  # (1, B, d_w)

  # rnn model 안에는 자체적으로 길이만큼 for문을 돌도록 구현
  # 길이 1짜리 input을 넣었으니
  # cell 하나 통과한 것과 똑같은 결과
  output, hidden = gru(input_emb, hidden)  # output: (1, B, d_h), hidden: (1, B, d_h)

  # V: vocab size
  output = output_layer(output)  # (1, B, V)
  probs, top_id = torch.max(output, dim=-1)  # probs: (1, B), top_id: (1, B)

  print("*" * 50)
  print(f"Time step: {t}")
  print(output.shape)
  print(probs.shape)
  print(top_id.shape)

  input_id = top_id.squeeze(0)  # (B)

**************************************************
Time step: 0
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 1
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 2
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 3
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 4
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 5
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 6
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 7
torch.Size([1, 10, 100])
torch.Si

`max_len`만큼의 for 문을 돌면서 모든 결과물의 모양을 확인했지만 만약 종료 조건(예를 들어 문장의 끝을 나타내는 end token 등)이 되면 중간에 생성을 그만둘 수도 있습니다.

### **양방향 및 여러 layer 사용**

이번엔 양방향 + 2개 이상의 layer를 쓸 때 얻을 수 있는 결과에 대해 알아봅니다.

In [13]:
num_layers = 2
num_dirs = 2
dropout=0.1

gru = nn.GRU(
    input_size=embedding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    dropout=dropout,
    bidirectional=True if num_dirs > 1 else False
)

Bidirectional이 되었고 layer의 개수가 $2$로 늘었기 때문에 hidden state의 shape도 `(4, B, d_h)`가 됩니다.

In [14]:
# d_w: word embedding size, num_layers: layer의 개수, num_dirs: 방향의 개수
batch_emb = embedding(batch)  # (B, L, d_w)
h_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (num_layers * num_dirs, B, d_h) = (4, B, d_h)

# 4의 의미
# 첫번째 layer의 forward dir, backward dir
# 두번째 layer의 forward dir, backward dir
# 이 순서대로 붙어서 들어가 있다

packed_batch = pack_padded_sequence(batch_emb.transpose(0, 1), batch_lens)

packed_outputs, h_n = gru(packed_batch, h_0)
print(packed_outputs)
print(packed_outputs[0].shape)
print(h_n.shape)

PackedSequence(data=tensor([[-0.0957, -0.0166,  0.0554,  ...,  0.1023,  0.1899,  0.0544],
        [-0.0686,  0.0301, -0.1713,  ...,  0.0457,  0.0056,  0.1121],
        [ 0.1350, -0.1133, -0.1373,  ..., -0.0269, -0.0185,  0.1510],
        ...,
        [ 0.0790, -0.0945, -0.0058,  ...,  0.0794,  0.1932, -0.0285],
        [-0.0788,  0.1639, -0.2551,  ...,  0.1163,  0.0133,  0.0524],
        [-0.0872,  0.2172, -0.2684,  ...,  0.0447,  0.0283,  0.0189]],
       grad_fn=<CatBackward>), batch_sizes=tensor([10, 10, 10, 10, 10,  9,  7,  7,  6,  6,  5,  5,  5,  5,  5,  4,  4,  3,
         1,  1]), sorted_indices=None, unsorted_indices=None)
torch.Size([123, 1024])
torch.Size([4, 10, 512])


In [15]:
outputs, output_lens = pad_packed_sequence(packed_outputs)

print(outputs.shape)  # (L, B, num_dirs*d_h)
print(output_lens)

torch.Size([20, 10, 1024])
tensor([20, 18, 18, 17, 15, 10,  8,  6,  6,  5])


각각의 결과물의 shape는 다음과 같습니다.

`outputs`: `(max_len, batch_size, num_dir * hidden_size)`  
`h_n`: `(num_layers*num_dirs, batch_size, hidden_size)`

In [16]:
batch_size = h_n.shape[1]

# view로 layer와 dir 분리
print(h_n.view(num_layers, num_dirs, batch_size, hidden_size))
print(h_n.view(num_layers, num_dirs, batch_size, hidden_size).shape)

tensor([[[[ 0.0522,  0.4401,  0.4628,  ...,  0.2844, -0.4757,  0.3456],
          [-0.1115, -0.0468,  0.2259,  ..., -0.4771, -0.1255,  0.1242],
          [ 0.0437, -0.1632, -0.2602,  ...,  0.1415, -0.3525, -0.0825],
          ...,
          [ 0.3407,  0.3344,  0.0991,  ...,  0.0916, -0.0726,  0.1108],
          [ 0.2581, -0.0370, -0.2818,  ..., -0.3138,  0.0598, -0.2039],
          [-0.1595,  0.1253, -0.1541,  ...,  0.0755,  0.1521,  0.2902]],

         [[ 0.2018,  0.0676, -0.1078,  ..., -0.3864, -0.3038,  0.0344],
          [ 0.0385,  0.1270,  0.3818,  ...,  0.1379,  0.0512,  0.2351],
          [ 0.1968,  0.3757,  0.2424,  ..., -0.1083, -0.0856,  0.1605],
          ...,
          [-0.2493, -0.0898,  0.0983,  ...,  0.2959,  0.1349,  0.4911],
          [ 0.3874,  0.0045, -0.3001,  ..., -0.3361,  0.3570,  0.2867],
          [-0.0103,  0.4331, -0.3085,  ..., -0.1546, -0.2151, -0.2990]]],


        [[[-0.0872,  0.2172, -0.2684,  ...,  0.0392, -0.0930,  0.1418],
          [-0.1356,  0.0282,